In [ ]:
import os
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.utils import class_weight
from scipy import sparse


import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler


import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
# 1. Load data
CSV_PATH = "pe_features_extended.csv"   # đổi theo đường dẫn của bạn
df = pd.read_csv(CSV_PATH)

print("Samples per class:\n", df['label'].value_counts())

In [ ]:
# 2. Preprocess / Feature handling
# create imports_text from import_0..import_9 if present
import_cols = [c for c in df.columns if c.startswith("import_")]
if import_cols:
    df['imports_text'] = df[import_cols].fillna("").astype(str).apply(
        lambda row: " ".join([t for t in row if t and t != 'nan']), axis=1
    )

In [ ]:
# drop obviously useless columns
drop_like = [c for c in df.columns if c.lower() in ('md5','filename','filepath','file_path')]
df = df.drop(columns=[c for c in drop_like if c in df.columns])

In [ ]:
# Drop object columns except 'imports_text' and 'label'
obj_cols = df.select_dtypes(include='object').columns.tolist()
for c in obj_cols:
    if c not in ('imports_text', 'label'):
        df = df.drop(columns=[c])

In [ ]:
# Map label to binary
y = (df['label'].str.lower() == 'malware').astype(int).values
df = df.drop(columns=['label'])

In [ ]:
# Numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

In [ ]:
# If imports_text does not exist, create empty
if 'imports_text' not in df.columns:
    df['imports_text'] = ""

In [ ]:
# ColumnTransformer: numeric pipeline + text vectorizer
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [ ]:
MAX_IMPORT_FEATURES = 500   # tune this
preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipeline, numeric_cols),
    ('imp', CountVectorizer(max_features=MAX_IMPORT_FEATURES, token_pattern=r"(?u)\b\w+\b"), 'imports_text')
], remainder='drop', sparse_threshold=0.3)

In [ ]:
# 3. Train/test split (stratified)
X_train_df, X_test_df, y_train, y_test = train_test_split(
    df, y, test_size=0.2, stratify=y, random_state=42
)

# Fit preprocessor on train
preprocessor.fit(X_train_df)

# transform to arrays (may be sparse)
X_train_trans = preprocessor.transform(X_train_df)
X_test_trans  = preprocessor.transform(X_test_df)

num_features = numeric_cols
text_features = preprocessor.named_transformers_['imp'].get_feature_names_out(['imports_text'])
all_feature_names = np.concatenate([num_features, text_features])

if sparse.issparse(X_train_trans):
    X_train_dense = X_train_trans.toarray()
    X_test_dense  = X_test_trans.toarray()
else:
    X_train_dense = X_train_trans
    X_test_dense  = X_test_trans
    
X_train_df_processed = pd.DataFrame(X_train_dense, columns=all_feature_names)
X_test_df_processed  = pd.DataFrame(X_test_dense, columns=all_feature_names)

X_train_df_processed['label'] = y_train
X_test_df_processed['label']  = y_test

X_train_df_processed.to_csv("train_preprocessed.csv", index=False)
X_test_df_processed.to_csv("test_preprocessed.csv", index=False)
print("[+] Saved preprocessed train/test CSV")

# Convert to dense if small enough, else keep sparse (RandomForest requires dense)
# Here we convert to dense — if you have memory issues, reduce MAX_IMPORT_FEATURES
if sparse.issparse(X_train_trans):
    X_train = X_train_trans.toarray()
    X_test  = X_test_trans.toarray()
else:
    X_train = X_train_trans
    X_test  = X_test_trans

print("Feature matrix shape:", X_train.shape)

In [ ]:
# 4. Baseline model: RandomForest with class_weight='balanced'
rf = RandomForestClassifier(n_estimators=200, n_jobs=-1, class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)[:,1]

In [ ]:
cm_rf = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm_rf, annot=True, fmt="d", cmap="Blues", xticklabels=["Benign","Malware"], yticklabels=["Benign","Malware"])
plt.title("RandomForest Confusion Matrix")
plt.ylabel("True")
plt.xlabel("Predicted")
plt.tight_layout()
plt.show()

print("=== RandomForest Report ===")
print(classification_report(y_test, y_pred, digits=4))
print("ROC AUC:", roc_auc_score(y_test, y_prob))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
# Save rf model + preprocessor
joblib.dump(rf, "rf_pe_model.joblib")
joblib.dump(preprocessor, "preprocessor.joblib")
print("[+] Saved RandomForest model and preprocessor.")

In [ ]:
# Prepare tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
X_test_t  = torch.tensor(X_test, dtype=torch.float32)
y_test_t  = torch.tensor(y_test, dtype=torch.float32)

# Sample weights to handle imbalance (WeightedRandomSampler)
class_sample_count = np.bincount(y_train)
weights = 1.0 / class_sample_count
samples_weight = weights[y_train]
samples_weight = torch.from_numpy(samples_weight.astype(np.float32))
sampler = WeightedRandomSampler(samples_weight, num_samples=len(samples_weight), replacement=True)

train_dataset = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler)
test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), batch_size=256, shuffle=False)

In [ ]:
# Simple MLP
class MLP(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )
    def forward(self, x):
        return self.net(x).squeeze(1)

In [ ]:
model = MLP(X_train.shape[1]).to(device)

In [ ]:
# use pos_weight to penalize false negatives
pos = max(1, int(y_train.sum()))
neg = len(y_train) - pos
pos_weight = torch.tensor([neg / pos], dtype=torch.float32).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Train loop
EPOCHS = 10
history = {"loss": [], "roc": []}
for epoch in range(1, EPOCHS+1):
    model.train()
    total_loss = 0.0
    for xb, yb in train_loader:
        xb = xb.to(device); yb = yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
    avg_loss = total_loss / len(train_loader.dataset)

    # eval on test
    model.eval()
    preds = []
    probs = []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb = xb.to(device)
            out = model(xb)
            prob = torch.sigmoid(out).cpu().numpy()
            pred = (prob >= 0.5).astype(int).ravel()
            probs.extend(prob.ravel().tolist())
            preds.extend(pred.tolist())
    roc = roc_auc_score(y_test, probs)
    history["loss"].append(avg_loss)
    history["roc"].append(roc)
    print(f"Epoch {epoch:02d} | loss={avg_loss:.4f} | test ROC={roc:.4f}")

In [ ]:
# 5c. Vẽ learning curves
# -------------------------
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(range(1,EPOCHS+1), history["loss"], marker='o')
plt.title("MLP Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.subplot(1,2,2)
plt.plot(range(1,EPOCHS+1), history["roc"], marker='o', color="green")
plt.title("MLP Test ROC AUC")
plt.xlabel("Epoch")
plt.ylabel("ROC AUC")

plt.tight_layout()
plt.show()

In [ ]:
# -------------------------
# 5d. Confusion Matrix cho MLP
# -------------------------
cm_mlp = confusion_matrix(y_test, preds)
plt.figure(figsize=(5,4))
sns.heatmap(cm_mlp, annot=True, fmt="d", cmap="Oranges", xticklabels=["Benign","Malware"], yticklabels=["Benign","Malware"])
plt.title("MLP Confusion Matrix")
plt.ylabel("True")
plt.xlabel("Predicted")
plt.tight_layout()
plt.show()

In [ ]:
# Final evaluation
print("=== PyTorch MLP Report ===")
print(classification_report(y_test, preds, digits=4))
print("ROC AUC:", roc_auc_score(y_test, probs))
print("Confusion matrix:\n", confusion_matrix(y_test, preds))

In [ ]:
# Save model and preprocessor
torch.save(model.state_dict(), "pe_mlp.pt")
joblib.dump(preprocessor, "preprocessor.joblib")  # already saved but safe
print("[+] Saved PyTorch model and preprocessor.")